In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

In [ ]:
tg = pd.read_csv("/kaggle/input/nsetataglobal/NSE-TATAGLOBAL11.csv")

In [ ]:
tg.head()

In [ ]:
tg.tail()

In [ ]:
tg.describe()

In [ ]:
nan = [(c, tg[c].isnull().mean()*100) for c in tg]
nan = pd.DataFrame(nan, columns = ["Column_name", "percentage"])
nan

In [ ]:
srt = tg.sort_values(by = "Date")
srt.head()

In [ ]:
srt.reset_index(inplace = True)

In [ ]:
srt.head()

In [ ]:
plt.figure(figsize = (10, 7))
plt.plot(srt["Date"], srt["Close"])

In [ ]:
close_srt = srt[["Close"]]
close_srt

In [ ]:
scaler = MinMaxScaler(feature_range = (0, 1))
close_srt = scaler.fit_transform(np.array(close_srt).reshape(-1,1))
close_srt

In [ ]:
train_size = int(len(close_srt)*0.7)
test_size = len(close_srt) - train_size
train_data, test_data = close_srt[0:train_size,:],close_srt[train_size:len(close_srt),:1]

In [ ]:
train_data.shape

In [ ]:
test_data.shape

In [ ]:
def create_dataset(dataset, time_step = 1):
    dataX, dataY = [],[]
    for i in range(len(dataset) - time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
print(X_train.shape), print(y_train.shape)

In [ ]:
print(X_test.shape), print(y_test.shape)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (100, 1)))
model.add(LSTM(50, return_sequences = True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss = "mean_squared_error", optimizer = "adam")

In [ ]:
model.summary()

In [ ]:
model.fit(X_train, y_train, validation_split = 0.1, epochs = 60, batch_size =64, verbose =1)

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

In [ ]:
import math
from sklearn.metrics import mean_squared_error

In [ ]:
math.sqrt(mean_squared_error(y_train, train_predict))

In [ ]:
math.sqrt(mean_squared_error(y_test, test_predict))

In [ ]:
look_back = 100

trainpredictplot = np.empty_like(close_srt)
trainpredictplot[:,:] = np.nan
trainpredictplot[look_back:len(train_predict) + look_back, :] = train_predict

testpredictplot = np.empty_like(close_srt)
testpredictplot[:,:] = np.nan
testpredictplot[len(train_predict)+(look_back*2)+1:len(close_srt)-1,:] = test_predict

plt.figure(figsize = (10,7))
plt.plot(scaler.inverse_transform(close_srt))
plt.plot(trainpredictplot)
plt.plot(testpredictplot)
plt.show()

In [ ]:
len(test_data)

In [ ]:
pred_input = test_data[271:].reshape(1,-1)
pred_input.shape

In [ ]:
temp_input = list(pred_input)
temp_input = temp_input[0].tolist()

In [ ]:
temp_input

In [ ]:
lst_output = []
n_steps = 100
i = 0
while(i<30):
    if(len(temp_input)>100):
        pred_input = np.array(temp_input[1:])
        print("{} day input {}".format(i, pred_input))
        pred_input = pred_input.reshape(1, -1)
        pred_input = pred_input.reshape((1, n_steps, 1))
        yhat = model.predict(pred_input, verbose =0)
        print("{} day input {}".format(i, yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input = temp_input[1:]
        lst_output.extend(yhat.tolist())
        i = i+1
    else:
        pred_input = pred_input.reshape((1, n_steps, 1))
        yhat = model.predict(pred_input, verbose = 0)
        print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i = i + 1
print(lst_output)

In [ ]:
day_new = np.arange(1, 101)
day_pred = np.arange(101, 131)

In [ ]:
len(close_srt)

In [ ]:
plt.figure(figsize = (10,7))
plt.plot(day_new, scaler.inverse_transform(close_srt[1135:]))
plt.plot(day_pred, scaler.inverse_transform(lst_output))

In [ ]:
df3 = close_srt.tolist()
df3.extend(lst_output)

In [ ]:
print(len(df3))

In [ ]:
plt.figure(figsize =  (10,7))
plt.plot(df3[1135:])

In [ ]:
df3 = scaler.inverse_transform(df3).tolist()

In [ ]:
plt.figure(figsize = (10,7))
plt.plot(df3)